In [34]:
import json

with open('credentials.json') as f:
    data = json.load(f)
    api_key = data['api_key']
    openai_api_key = data['openai_api_key']

In [3]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=api_key)


/home/mchone/Documents/projects/pinecone-notebook/venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:

with open("emoji.txt") as f:
    emoji = [line.split("#")[1].strip().split(" ", 2) for line in f if "fully-qualified" in line and not line.startswith("#")]

print(len(emoji))

3773


In [5]:
e = emoji[0]
print(e)

print(float(e[1].removeprefix("E")))


['😀', 'E1.0', 'grinning face']
1.0


In [6]:
index = pc.Index("emoji")


In [35]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=openai_api_key,
)

In [7]:
vectors = []
i = 0
for e in emoji:
    if len(e) != 3:
        continue
    embedding = client.embeddings.create(input = [e[2]], model="text-embedding-3-small").data[0].embedding
    vectors.append({"id": e[0], "values": embedding, "metadata": { "edition": float(e[1].removeprefix("E")), "name": e[2] }},)
    print(i)
    i = i + 1
    
print(vectors[0])
print(len(vectors))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [31]:
for v in vectors:
    v["metadata"]["emoji"] = v["id"]
    v["id"] = v["metadata"]["name"]
    v["id"] = v["id"].replace("’", "-").replace("ñ", "n").replace("“", "\"").replace("”", "\"").replace("Å", "A").replace("é", "e").replace("ô", "o").replace("ç", "c").replace("ã", "a").replace("í", "i").replace("ü", "u")
print(vectors[0])

{'id': 'grinning face', 'values': [0.004887338, -0.024229012, 0.025765823, 0.027164184, -0.011477629, -0.015631175, 0.030874684, 0.026250403, -0.034889776, -0.021736884, 0.04535671, 0.020172384, 0.02366136, -0.035720486, 0.00022217138, 0.038738728, -0.049759466, -0.012785995, -0.0013230771, 0.01295906, 0.031760775, 0.011712996, 0.0037901097, 0.03148387, 0.017957158, 0.046547394, 0.013935143, -0.044941355, 0.027967202, -0.0036101227, -0.07232706, -0.04895645, 0.006635288, -0.07515147, 0.02868715, 0.029213266, 0.003540897, -0.0025907736, -0.015866542, -0.020490821, -0.005610747, -0.06806275, 0.015354271, 0.02078157, 0.027704144, -0.022941412, 0.008639373, 0.00863245, 0.044913664, 0.058537293, -0.016434193, 0.0006377422, -0.0027274946, 0.052390046, -0.05200238, 0.011789145, 0.013305189, 0.0058011175, -0.026610378, 0.016503418, 0.021833802, -0.020476976, 0.027468776, 0.033892926, -0.010660765, -0.03245303, -0.023439838, 0.023758277, -0.054134537, 0.042781513, 0.0018171759, -0.0029005588, 0

In [32]:
index.upsert(vectors=vectors[3699:], batch_size=100, show_progress=True)

Upserted vectors: 100%|██████████| 73/73 [00:01<00:00, 71.32it/s]


{'upserted_count': 73}

In [33]:
with open("my.json","w") as f:
    json.dump(vectors,f)